In [ ]:
# install the packages
! pip install diffusers -q
! pip install accelerate -q

In [ ]:
# import libraries
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid
import torch

# define the pipeline object
pipe = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16").to("cuda")

# load the image and mask
img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

input_image = load_image(img_url).resize((1024, 1024))
mask_image = load_image(mask_url).resize((1024, 1024))

# defining variables for inference pipeline
prompt = "a cute baby lion sitting on a park bench"
negative_prompt = "bad anatomy, deformed, ugly, disfigured"
generator = torch.Generator(device="cuda").manual_seed(0)

# inpainting pipeline creation
image = pipe(
  prompt=prompt,
  negative_prompt=negative_prompt,
  image=input_image,
  mask_image=mask_image,
  guidance_scale=20.0, # affects the alignment between the text prompt and the generated image.
  num_inference_steps=100,  # steps between 15 and 30 work well for us
  strength=0.99,  # make sure to use `strength` below 1.0
  generator=generator,
  padding_mask_crop=32 #Crops masked area + padding from image and mask.
).images[0]

#disply the output
make_image_grid([input_image,mask_image,image], rows=1, cols=3)